In [18]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [19]:
def resample_to_constant_volume(df, volume_per_candle, convertTime= False):
    # sort by time
    #df = df.sort_values(by='time')
    # create cumulative volume column
    df['cumulative_volume'] = df['volume'].cumsum()
    
    #print(len(df))
    # initialize the new dataframe to hold resampled data
    resampled_df = pd.DataFrame(columns=['open', 'high', 'low', 'close','volume'])
    
    start_index = 0
    end_index = 0
    cumulative_volume = 0
    for i in range(len(df)):
        cumulative_volume += df.iloc[i]['volume']
        end_index = i
        if cumulative_volume >= volume_per_candle : # make function dynamic in next round
            candle_df = df.iloc[start_index:end_index+1]
            open_price = candle_df.iloc[0]['open']
            high_price = candle_df['high'].max()
            low_price = candle_df['low'].min()
            close_price = candle_df.iloc[-1]['close']
            date_end = df.iloc[start_index]["date"]
            #x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
            #date_end = x.strftime(f"%Y-%m-%d %H:{5*int(x.minute/5)}:%S")
            if convertTime:
                mask = df.iloc[end_index]["date"].split("+")[0]
                x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
                ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
                temp = ("0"+str(5*int(x.minute/5)))
                date_end = x.strftime(f"%Y-%m-%d %H:{temp[-2:]}:%S")
            """resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                'close': close_price, 'volume': cumulative_volume,
                                                 "date_end": date_end}, 
                                                 ignore_index=True)"""
            for i in range(int(cumulative_volume/volume_per_candle)):
                resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                    'close': close_price, 'volume': cumulative_volume,
                                                     "date_end": date_end}, 
                                                     ignore_index=True)
            cumulative_volume = 0
            start_index = end_index + 1
    
    if start_index < len(df):
        candle_df = df.iloc[start_index:len(df)]
        open_price = candle_df.iloc[0]['open']
        high_price = candle_df['high'].max()
        low_price = candle_df['low'].min()
        close_price = candle_df.iloc[-1]['close']
        volume = sum(candle_df["volume"])
        date_end = df.iloc[start_index]["date"]
        #x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
        ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
        #date_end = x.strftime(f"%Y-%m-%d %H:{5*int(x.minute/5)}:%S")
        if convertTime:
            mask = df.iloc[end_index]["date"].split("+")[0]
            x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
            temp = ("0"+str(5*int(x.minute/5)))
            date_end = x.strftime(f"%Y-%m-%d %H:{temp[-2:]}:%S")
        resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price, 'volume': volume,"date_end": str(date_end)}, ignore_index=True)

    return resampled_df

In [20]:
from datetime import timedelta
def is_body_size_greater_than_nx_average(dataframe, index,atr_multiple,look_back):
    body_sizes = (dataframe['close'] - dataframe['open']).loc[index - look_back:index].abs()
#     print(body_sizes,"efb")
    average_body_size = body_sizes.mean()
    print(average_body_size)
    current_body_size = abs(dataframe['close'].loc[index] - dataframe['open'].loc[index])
    return current_body_size > atr_multiple * average_body_size


def imbalance_from_volume(df, dateTime_index_to_check):
    
    ## df is 1min    
    """" Takes volume sampled data and returns a list of imbalanced candlesticks """
    #df = df.reset_index()
    ##index of the dateTime
    #print(data)
    ind = df[df["date"] == dateTime_index_to_check].index[0]
    #normal = df.iloc[0].index[0]
    #print(df)
    # df = df[:ind+1]
    # print(df)
    
    ##end  index to be the +5 min of dateTime index
    x =  pd.to_datetime(datetime.strptime(dateTime_index_to_check, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=5))
    #temp = ("0"+str(5*int((x.minute+5)/5)))
    endTime = x.strftime(f"%Y-%m-%d %H:%M:%S")
    # ind2 = df[df["date"] == endTime].index[0]
    # df = df[:ind+1]
    #print(endTime)
    end_index = (df[df["date"] == endTime].index)
    if len(end_index):
        end_index=end_index[0] ## corner case 15:30
    else:
        return False
    
    #print(data["volume"].rolling(window=80000).mean()[ind])
    #vol_resampled_data = resample_to_constant_volume(df[ind-2000:end_index],df["volume"].rolling(window=80000).mean()[ind]*1.7)
    
    volume_per_candle = df["volume"].rolling(window=8000).quantile(0.95).loc[ind]*2.5
    vol_resampled_data1 = resample_to_constant_volume(df.loc[ind-2000:end_index], volume_per_candle ,convertTime = True)
    #print((df.loc[ind-20000:end_index], volume_per_candle))
    #print(vol_resampled_data1)
    
    index_in_vol = vol_resampled_data1.index[-1]#vol_resampled_data1[vol_resampled_data1["date_end"] == dateTime_index_to_check].index[0]

#TODO modified this to return volume candle
    # if ((vol_resampled_data1[["close" , "open", "low"]] == vol_resampled_data1[["close" , "open", "low"]].loc[index_in_vol].shift()).all()
    #     & is_body_size_greater_than_nx_average(vol_resampled_data1, index_in_vol ,2,30)) :

    if(((vol_resampled_data1['volume'][index_in_vol])/volume_per_candle) > 2 or (is_body_size_greater_than_nx_average(vol_resampled_data1, index_in_vol ,2,30))):
        return True
    else :
        return False

In [21]:
def is_body_size_greater_than_3x_average(dataframe, index,atr_multiple,look_back):   #price-time 
    body_sizes = (dataframe['close'] - dataframe['open']).loc[index - look_back:index].abs()
#     print(body_sizes)
    average_body_size = body_sizes.mean()
#     print(average_body/_size,"Wrv__________")
    current_body_size = abs(dataframe['close'].loc[index] - dataframe['open'].loc[index])
#     print(current_body_size,"current")
    return current_body_size > atr_multiple * average_body_size

def imbalance_zone(df,dateTime_index_to_check,look_back=30):   #return imbalance zone ( price levels)
    imbalance_candle = df[df["date"] == dateTime_index_to_check].index[0]
    verify_candle  = imbalance_candle+1
    current_candle = imbalance_candle+2
    
    for i in range(look_back):
        
        if is_body_size_greater_than_3x_average(df,imbalance_candle,1.8,look_back):
            upper = min(df["low"].loc[imbalance_candle-30:imbalance_candle])
            lower = df["high"].loc[verify_candle]
            print(upper,lower)

            if (upper-lower) > 0 :
                return [ lower , upper ]  ## test in future the difference of lines more than some % of body

            else :
                return [0 , 0]
        else:
            return [0 , 0]


In [22]:
from collections import defaultdict

def to_intervals(data,n):   # to convert it n minute candle 
    # Read in the data from the text file
    data
    data = data.reset_index()
    data = np.array(data)
    """data = []
    name = "data_2017.txt" #input()
    with open(name, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            # Ignore the first column (stock name)
            data.append(row[1:])"""

    # Convert the data to n-minute intervals
    interval_data =  defaultdict(list)
    for row in data:
        date = row[1].split()[0]
        time = row[1].split()[1].split("+")[0][:-3]
        if time == "09:08:00":
            continue

        open_price = float(row[2])
        high_price = float(row[3])
        low_price = float(row[4])
        close_price = float(row[5])
        volume = float(row[6])
        # Extract the interval from the time and use it as the key in the dictionary
        interval = int(time[-2:]) // n
        interval = f"{time[:2]}:{interval * n:02d}"
        interval = str(date) +" " + interval + ":00"
        interval_data[interval].append((date, time, open_price, high_price, low_price, close_price, volume))

    # Calculate the final open, high, low, and close prices for each interval
    final_prices = {}
    for interval, prices in interval_data.items():
        # Unpack the tuples into separate lists for each price type
        dates, times, opens, highs, lows, closes, volume = zip(*prices)
        # The final open price is the same as the first open price in the interval
        final_open = opens[0]

        # The final high price is the highest of all the high prices in the interval
        final_high = max(highs)
        # The final low price is the lowest of all the low prices in the interval
        final_low = min(lows)
        # The final close price is the same as the last close price in the interval
        final_close = closes[-1]
        # The final volume will be the sum of all volumes of the candels
        final_volume = sum(volume)
        # Store the final prices in a tuple in the dictionary
        final_prices[interval] = (dates[0], times[0], final_open, final_high, final_low, final_close, final_volume)
    
    ws = []
    # Write the data rows
    for interval, final in final_prices.items():
        date, time, open_price, high_price, low_price, close_price, volume = final
        ws.append([date, time, interval, open_price, high_price, low_price, close_price, volume])
    return pd.DataFrame(ws,columns = ["Date", "Time", "date", "open", "high", "low", "close","volume"])
    

def handle_data(context,data):
    ### check on index n-1::
    ## Fetch n-1 candles volume candles:::
    #data = data.reset_index()
    
    ## exit prev conditions
    if context["position"] == True:
        if data["close"]>=sum(context["priceLevels"])/2:
            context["profit"] += data["close"].iloc[-1] - context["buy"]
            context["profitList"].append(context["profit"])
            context["position"]  = 0
            context["trades"] += 1
        elif data["close"]<=context["stoploss"]:
            context["losses"] += 1
            context["position"] = 0
            context["profit"] += data["close"].iloc[-1] - context["buy"]
            context["profitList"].append(context["profit"])
            context["trades"] += 1
        return
        
    data_5min = to_intervals(data[['date',"open",'high',"low","close","volume"]].iloc[-5000:],5)
    
    candleToCheck = data_5min["date"].iloc[-2]
    
    ## apply volume candle conditions
#     print(candleToCheck)
    if not imbalance_from_volume(data,candleToCheck):
        return
    print("imbalance Found in volume!!")
    ## apply price conditions 
    ## between previous 30 5 minute candles low, current candle high -- range of body of imbalance greater than threshold
    ## 5 min data
    p1,p2 = imbalance_zone(data_5min, candleToCheck , 30) ##zone funciton....
    print(p1,p2)
    if p1==p2:
        return
    context["priceLevels"] = [p1,p2]
    # strategy_trade(context,data_5min,candleToCheck) #TODO send volume data
    context["imbalanceCandles"].append(candleToCheck)
    next_candle_strategy_trade(context , data_5min)

def buy(context,price):
    context["position"] = True
    context["buy"] = price
    context['stoploss'] = price - 0.02*price   # to be change 

def next_candle_strategy_trade( context , data):
    print("buy order")
    buy(context,data["close"].iloc[-1])


def strategy_trade(context,df,candleToCheck):
    data = df.iloc[-50:]
    data["mid"] = (data["high"]+data["low"])/2
    data["ao"] = data["mid"].rolling(window = 5).mean() - data["mid"].rolling(window = 34).mean()
    signals = data[(data["ao"].shift(4)< data["ao"].shift(3)) & (data["ao"].shift(3)< data["ao"].shift(2)) & (data["ao"].shift(2) > data["ao"])]
    if len(signal):
        ind = signal.index[-1]
        datetime_ = signal["datetime"].loc[ind]  #latest or not ==== send data only after signal
        # datetime.datetime.strptime(datetime_,"")>candleToCheck:

        #TODO 

        buy(context,data["open"].iloc[-1])
        
    #data[data["ao"].shift(3)< data["ao"].shift(2) and data["ao"].shift(2)< data["ao"].shift(1) and data["ao"].shift(1) > data["ao"]]

In [37]:
from datetime import datetime
def ss(x):
    return datetime.strptime(x,"%Y-%m-%d %H:%M:%S+05:30").strftime("%Y-%m-%d %H:%M:%S")
def nifty_data(x):
    #print(x)
    #print(x[1])
    return datetime.strptime(str(x[1]),"%Y%m%d").strftime("%Y-%m-%d")+" "+x[2]+":00"
    #return datetime.strptime(x,"%Y-%m-%d %H:%M:%S+05:30").strftime("%Y-%m-%d %H:%M:%S")

#data = pd.read_csv("ACC_minute_data_with_indicators.csv", usecols = ["date","open","high","low","close","volume"]).iloc[150971:554674]
data = pd.read_csv("ACC_minute_data_with_indicators.csv")
# data.columns = ["a","d","t","open","high","low","close","volume"]
# data["date"]=list(map(nifty_data,np.array(data)))
# data = data[["date","open","high","low","close","volume"]]
# print("data_read")
#data["date"]=list(map(ss,data["date"]))
data1 = data[['date' , 'open' , 'high', 'low' , 'close' , 'volume']]

data1.head()

,date,open,high,low,close,volume
0,2015-02-02 10:18:00+05:30,1545.0,1545.00,1543.1,1544.0,220
1,2015-02-02 10:19:00+05:30,1544.0,1544.00,1543.0,1543.7,29
2,2015-02-02 10:20:00+05:30,1543.7,1543.70,1542.5,1543.0,28
3,2015-02-02 10:21:00+05:30,1543.0,1543.00,1542.3,1542.3,125
4,2015-02-02 10:22:00+05:30,1541.1,1542.15,1541.1,1542.1,113


In [38]:
#data from 2017 to 2021
# data1 = data[['date' , 'open' , 'high', 'low' , 'close' , 'volume']]
data1 = data1.iloc[249971-80000 :554674]
data1.head()

,date,open,high,low,close,volume
169971,2017-01-25 11:10:00+05:30,1364.4,1364.40,1364.00,1364.40,232
169972,2017-01-25 11:11:00+05:30,1364.4,1364.40,1364.40,1364.40,693
169973,2017-01-25 11:12:00+05:30,1364.4,1366.50,1364.40,1366.50,740
169974,2017-01-25 11:13:00+05:30,1366.5,1367.75,1366.45,1367.40,1264
169975,2017-01-25 11:14:00+05:30,1367.4,1367.45,1367.40,1367.45,247


In [40]:
data1[data1['date'] == '2018-02-12 13:20:00+05:30']

,date,open,high,low,close,volume
259966,2018-02-12 13:20:00+05:30,1657.2,1658.0,1657.2,1658.0,76


In [42]:
#run 
#data = pd.read_csv("data.")
context = {}
context["profit"] = 0
context["profitList"] = []#.append(context["profit"])
context["losses"] = 0

context["position"]  = 0
context["trades"] = 0
context["buy"] = 0
context['stoploss'] = 0# price - 0.02*price   # to be change 
context["priceLevels"] = [0,0]
context["imbalanceCandles"] = []
i = 90000
while i<(len(data1)):
    #print(data.iloc[i-80000:i+1])
    if "15:25" in data1["date"].iloc[i] or "09:15" in data1["date"].iloc[i] or "09:08" in data1["date"].iloc[i]:
        i+=5
        continue
    #print(data["date"].iloc[i],"bakchodi")
#     if not False:
#         print(i)
    handle_data(context,data1.iloc[i-10000:i+1])
    i+=5


2018-02-12 13:20:00


IndexError: index 0 is out of bounds for axis 0 with size 0

In [142]:
i = 80270
handle_data(context,data.iloc[i-10000:i+1])

imbalance Found in volume!!
0 0


In [140]:
context

{'profit': 0,
 'profitList': [],
 'losses': 0,
 'position': 0,
 'trades': 0,
 'buy': 0,
 'stoploss': 0,
 'priceLevels': [0, 0],
 'imbalanceCandles': []}

In [81]:
ind = data[data["date"] == "2017-10-12 12:15:00"].index[0]
resample_to_constant_volume(data[ind-20000:ind+6], 6000 ,convertTime = True)

20006


,open,high,low,close,volume,date_end
0,1628.65,1629.40,1626.50,1627.85,6074,2019-05-02 12:00:00
1,1627.85,1627.85,1624.40,1624.40,6094,2019-05-02 12:00:00
2,1624.40,1626.00,1622.35,1624.75,6858,2019-05-02 12:10:00
3,1624.75,1624.75,1619.25,1619.70,9682,2019-05-02 12:15:00
4,1619.70,1622.00,1619.20,1620.40,6532,2019-05-02 12:20:00
...,...,...,...,...,...,...
5752,1571.15,1575.60,1571.00,1573.90,11845,2019-07-24 13:35:00
5753,1573.90,1575.90,1573.90,1574.70,8277,2019-07-24 13:40:00
5754,1574.55,1574.55,1571.00,1573.05,6657,2019-07-24 13:45:00
5755,1573.05,1573.10,1570.35,1571.75,6463,2019-07-24 13:50:00


In [202]:
data_5min = to_intervals(data,5)
    
candleToCheck = data_5min["date"].iloc[-2]

p1,p2 = imbalance_zone(data_5min, candleToCheck , 30)

In [211]:
data_15 = to_intervals(data,15)

In [127]:
for i in range(231001,231001+10000):
    print(is_body_size_greater_than_nx_average(data, i,2,30))

False
True
False
False
False
True
True
False
False
False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
True
False
False
True
False
False
False
False
True
True
False
False
True
False
False
False
False
True
True
False
False
False
False
False
True
True
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
True
False
False
True
True
True
Fa

In [79]:
data[data["date"] == "2017-10-12 12:15:00"].index[0]

230966

In [37]:
import warnings
warnings.filterwarnings("ignore")

In [44]:
data[-381972+231001+231001:].head(11)

,date,close,high,low,open,volume
231001,2017-10-12 12:50:00,1712.55,1714.05,1711.70,1712.70,1861
231002,2017-10-12 12:51:00,1714.60,1714.60,1711.05,1711.70,2102
231003,2017-10-12 12:52:00,1715.10,1717.80,1714.75,1714.75,1859
231004,2017-10-12 12:53:00,1714.30,1715.80,1714.30,1715.10,82
231005,2017-10-12 12:54:00,1713.00,1714.60,1713.00,1714.30,18
231006,2017-10-12 12:55:00,1714.60,1714.60,1712.25,1713.00,187
231007,2017-10-12 12:56:00,1712.95,1714.60,1712.95,1714.60,232
231008,2017-10-12 12:57:00,1712.90,1713.05,1712.90,1712.95,104
231009,2017-10-12 12:58:00,1712.35,1713.00,1712.00,1712.90,113
231010,2017-10-12 12:59:00,1713.05,1713.05,1712.10,1712.35,599


In [225]:
for i in (data_15.index):
    if (is_body_size_greater_than_3x_average(data_15, i,0.2,400)):
        upper = min(data_15["low"].loc[i-10:i])
        lower = data_15["high"].loc[i+1]
#         print("A")
        if (upper>lower):
            print(i,"__________________________________")
    #print(is_body_size_greater_than_nx_average(data, i ,2,30))

0    53.9
dtype: float64
53.899999999999636 Wrv__________
53.899999999999636 current
0    53.9
1     8.1
dtype: float64
30.999999999999545 Wrv__________
8.099999999999454 current
0    53.90
1     8.10
2     3.95
dtype: float64
21.98333333333297 Wrv__________
3.949999999999818 current
0    53.90
1     8.10
2     3.95
3     6.65
dtype: float64
18.149999999999636 Wrv__________
6.649999999999636 current
0    53.90
1     8.10
2     3.95
3     6.65
4     5.20
dtype: float64
15.559999999999672 Wrv__________
5.199999999999818 current
0    53.90
1     8.10
2     3.95
3     6.65
4     5.20
5    12.10
dtype: float64
14.983333333333121 Wrv__________
12.100000000000364 current
0    53.90
1     8.10
2     3.95
3     6.65
4     5.20
5    12.10
6     5.90
dtype: float64
13.685714285714182 Wrv__________
5.900000000000546 current
0    53.90
1     8.10
2     3.95
3     6.65
4     5.20
5    12.10
6     5.90
7     4.30
dtype: float64
12.512499999999818 Wrv__________
4.299999999999272 current
0    53.90
1  

In [204]:
data_5min

,Date,Time,date,open,high,low,close,volume
0,2017-01-02,09:16,2017-01-02 09:15:00,8212.00,8212.00,8168.80,8177.55,231.0
1,2017-01-02,09:20,2017-01-02 09:20:00,8177.85,8178.15,8164.40,8168.95,285.0
2,2017-01-02,09:25,2017-01-02 09:25:00,8169.60,8171.50,8157.90,8158.10,278.0
3,2017-01-02,09:30,2017-01-02 09:30:00,8157.10,8157.60,8141.00,8151.90,292.0
4,2017-01-02,09:35,2017-01-02 09:35:00,8151.45,8164.95,8151.45,8164.95,279.0
...,...,...,...,...,...,...,...,...
18998,2017-12-29,15:10,2017-12-29 15:10:00,10529.30,10532.20,10527.30,10530.90,283.0
18999,2017-12-29,15:15,2017-12-29 15:15:00,10530.40,10538.70,10530.40,10532.00,290.0
19000,2017-12-29,15:20,2017-12-29 15:20:00,10532.50,10534.00,10530.50,10533.50,284.0
19001,2017-12-29,15:25,2017-12-29 15:25:00,10533.30,10535.40,10529.00,10531.30,286.0


In [212]:
data_15

,Date,Time,date,open,high,low,close,volume
0,2017-01-02,09:16,2017-01-02 09:15:00,8212.00,8212.00,8157.90,8158.10,794.0
1,2017-01-02,09:30,2017-01-02 09:30:00,8157.10,8167.75,8141.00,8165.20,851.0
2,2017-01-02,09:45,2017-01-02 09:45:00,8165.05,8165.95,8153.25,8161.10,824.0
3,2017-01-02,10:00,2017-01-02 10:00:00,8161.25,8161.35,8152.60,8154.60,807.0
4,2017-01-02,10:15,2017-01-02 10:15:00,8154.85,8161.80,8153.45,8160.05,812.0
...,...,...,...,...,...,...,...,...
6662,2017-12-29,14:30,2017-12-29 14:30:00,10522.80,10532.00,10520.30,10530.20,831.0
6663,2017-12-29,14:45,2017-12-29 14:45:00,10529.90,10532.50,10515.50,10526.00,850.0
6664,2017-12-29,15:00,2017-12-29 15:00:00,10524.70,10532.20,10523.00,10530.90,847.0
6665,2017-12-29,15:15,2017-12-29 15:15:00,10530.40,10538.70,10529.00,10531.30,860.0


In [113]:
"12:12" in "2017-10-12 12:15:00"

False